In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import logging
import os
import re
import sys
from copy import deepcopy
from functools import cache, cached_property, partial
from inspect import signature
from itertools import chain
from pathlib import Path
from uuid import uuid4

import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import QuantLib as ql
import talib as ta
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from gioutils import ezutils as ez
from gioutils.blpw import BlpQuery
from gioutils.ezutils import bh, bhs
from gioutils.gui.pandas import millify
from gioutils.utils import (
    RepeatedTimer,
    date_from_offset,
    get_bbg_fut_chain_ticker,
    get_bday,
    get_n_from_code,
    get_next_n,
    parse_offset,
    parse_swap_tenor_expr,
    today,
    to_pickle,
    read_pickle
)
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from joblib import Parallel, delayed
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import ParameterGrid
from techi import api as ti
from termcolor import colored

from api.data.base import data_path, radar_path
from api.gui.pandas import DataGridGio, DFOutput, custom_styler_row_line, negative_red
from api.gui.params import bbg_params, fwd_params, spot_params
from api.gui.theme import bg_color, grid_style
from api.instruments import config as c
from api.instruments.cache import data_cache, get_key
from api.instruments.future import FutureChain, FutureFly, FutureSpread
from api.instruments.signals import EnsambleSignal, MACDHistCrossover, ZScoreThreshold
from api.instruments.signals2 import BaseSignal, MACDCustomSignal, get_crossover_signal
from api.instruments.strategies import FutureStrategy
from api.instruments.strategies2 import MACDCustomStrategy
from api.instruments.swap import Fly, RealSwap, RollingSwap, Spread
from api.instruments.utils import ez_bump, parse_dates, parse_number


# from bhsysapi import BHSysApi

In [3]:
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None)
c.USE_T_COST = True

In [24]:
bq = BlpQuery(timeout=50000).start()
# ba = BHSysApi(password="A&3jg5Aj")

[autoreload of api.instruments.swap failed: Traceback (most recent call last):
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 381, in update_generic
    update(a, b)
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 333, in update_class
    if update_generic(old_obj, new_obj):
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 381, in update_generic
    update(a, b)
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 293, in update_function
    setattr(old, name, getattr(new, name))
ValueError: __init__() req

In [5]:
notional = 1000
hist = "10y"
strategy_hist = "1y"

## Instruments Parameters

In [6]:
rates = [
    "er",
    "sfr",
    "ff",
    "sfi",
]

govt = ["tu", "fv", "ty", "us", "du", "oe", "rx", "ub", "ik"]

In [7]:
rates_single_legs = range(1, 12 + 1)
govt_single_legs = range(1, 2 + 1)

fly_legs = [
    (2, 3, 4),
    (2, 4, 6),
    (2, 5, 8),
    (2, 6, 10),
    (2, 7, 12),
    (3, 4, 5),
    (3, 5, 7),
    (3, 6, 9),
    (3, 7, 11),
    (3, 8, 13),
    (4, 6, 8),
    (4, 7, 9),
    (4, 7, 11),
    (4, 8, 12),
    (5, 8, 11),
    (5, 9, 13),
    (6, 9, 12),
]

spread_legs = [
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (2, 9),
    (3, 5),
    (3, 6),
    (3, 7),
    (3, 8),
    (3, 9),
    (3, 10),
    (3, 11),
    (3, 12),
    (4, 6),
    (4, 7),
    (4, 8),
    (4, 9),
    (4, 10),
    (4, 11),
    (4, 12),
    (5, 8),
    (5, 9),
    (5, 10),
    (5, 11),
    (5, 12),
    (6, 8),
    (6, 9),
    (6, 10),
    (6, 11),
    (6, 12),
    (7, 10),
    (7, 11),
    (7, 12),
    (8, 12),
]

In [8]:
watching = {"du1", "oe1", "rx1", "ik1", "er1"}

watching = {
    'du1': 1,
    'oe1': 1,
    'rx1': 1,
    'ub1': 2,
    'ik1': 1,
    "er1": 2,
    "er2": 1,
    "er3": 1,
    "er4": 1,
    "er5": 1,
    "er6": 1,
    "er7": 1,
    "er8": 1,
    "er9": 2,
    "er10": 3,
    "er11": 3,
    "er12": 1,
    "sfr1": 3,
    "sfr2": 2,
    "sfr3": 1,
    "sfr4": 2,
    "sfr5": 2,
    "sfr6": 1,
    "sfr7": 1,
    "sfr8": 2,
    "sfr9": 2,
    "sfr10": 2,
    "sfr11": 2,
    "sfr12": 1,
    "ff1": 2,
    "ff2": 3,
    "ff3": 3,
    "ff4": 1,
    "ff5": 1,
    "ff6": 2,
    "ff7": 2,
    "ff8": 1,
    "ff9": 2,
    "ff10": 2,
    "ff11": 2,
    "ff12": 2,
    "sfi1": 2,
    "sfi2": 3,
    "sfi3": 2,
    "sfi4": 1,
    "sfi5": 1,
    "sfi6": 1,
    "sfi7": 1,
    "sfi8": 1,
    "sfi9": 2,
    "sfi10":2,
    "sfi11": 3,
    "sfi12": 2,
}

In [9]:
stirs = read_pickle('top_stirs.pickle')

In [10]:
govt = read_pickle('top_govt.pickle')

In [11]:
ress = {**stirs, **govt}

In [25]:
strategies_orig = {key: ress[key].iloc[0] for key, code in watching.items()}
strategies = deepcopy(strategies_orig)

In [49]:
all_instruments = {s.instrument.ticker: s.instrument for k, s in strategies.items()}

In [50]:
start_hist = '10y'
for key, instrument in all_instruments.items():
    instrument.start_hist = start_hist

In [27]:
fields = {
    'px_last': 'quote', 
    'yld_ytm_mid': 'yld'
}

In [51]:
symbols = {instrument.symbol: instrument for key, instrument in all_instruments.items()}
# keys = [get_key(s.instrument, attr, keys=['start_hist', 'end_hist']) for key, s in strategies.items() for attr in ['quote', 'yld']]

In [29]:
hist = bq.bdh(
    securities=[*symbols.keys()],
    fields=fields,
    start_date=today() - parse_offset(start_hist),
)
hist = hist.set_index('date')

In [30]:
for symbol, instrument in symbols.items():
    for attr in fields.values():
        series = hist.query(f'security == "{symbol}"')[attr]
        series = instrument._rename_attr(series, attr)
        key = get_key(instrument, attr, keys=['start_hist', 'end_hist'])
        data_cache[key] = series

In [42]:
last = bq.bdp(
    securities=[*symbols.keys()],
    fields=fields,
)

In [43]:
for symbol, instrument in symbols.items():
    for attr in fields.values():
        value = last.query(f'security == "{symbol}"')[attr].squeeze()
        getattr(instrument, attr).iloc[-1] = value

In [44]:
for key, self in strategies.items():
    #print(key)
    self.run()

In [45]:
for key, self in strategies.items():
    # print(key)
    self.run(
        start=-1
    )

In [34]:
watch_codes = [
    1, 
    # 2, 
    # 3
]

In [46]:
for key, self in strategies.items():
    if (self._running or self._pnl.index[-1] == today()) and watching.get(key) in watch_codes:
        display(key.upper())
        last_trade = self._summary['trade'].iloc[-1]
        display(self._pnl.query(f"trade == {last_trade}"))

'SFI5'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-30      1   -1.00        95.83  95.83      -0.00       0.10   
2023-01-31      1   -1.00          NaN  95.87      -0.04       0.10   
2023-02-01      1   -1.00          NaN  95.87          0       0.10   

            take_profit  exit_price  notes     pnl       pv  \
date                                                          
2023-01-30         0.10         NaN    NaN -12.50k  -12.50k   
2023-01-31         0.10         NaN    NaN -87.50k -100.00k   
2023-02-01         0.10         NaN    NaN       0 -100.00k   

            quote_chg_per_trade  pv_per_trade  
date                                           
2023-01-30                -0.00       -12.50k  
2023-01-31                -0.04      -100.00k  
2023-02-01                -0.04      -100.00k

'SFI6'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-27      1   -1.00        96.12  96.12      -0.00       0.10   
2023-01-30      1   -1.00          NaN  96.08       0.05       0.10   
2023-01-31      1   -1.00          NaN  96.11      -0.03       0.10   
2023-02-01      1   -1.00          NaN  96.12      -0.02       0.10   

            take_profit  exit_price  notes     pnl      pv  \
date                                                         
2023-01-27         0.10         NaN    NaN -12.50k -12.50k   
2023-01-30         0.10         NaN    NaN 125.00k 112.50k   
2023-01-31         0.10         NaN    NaN -87.50k  25.00k   
2023-02-01         0.10         NaN    NaN -37.50k -12.50k   

            quote_chg_per_trade  pv_per_trade  
date                                           
2023-01-27                -0.00       -12.50k  
2023-01-30                 0.05       112.50k  
2023-01-31                 0.01        25.00k  
2023-02-01                -0.00       -12.50k

'SFI8'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-27      6   -1.00        96.51  96.51      -0.00       0.11   
2023-01-30      6   -1.00          NaN  96.47       0.04       0.11   
2023-01-31      6   -1.00          NaN  96.50      -0.03       0.11   
2023-02-01      6   -1.00          NaN  96.53      -0.04       0.11   

            take_profit  exit_price                               notes  \
date                                                                      
2023-01-27         0.11         NaN                                 NaN   
2023-01-30         0.11         NaN                                 NaN   
2023-01-31         0.11         NaN                                 NaN   
2023-02-01         0.11       96.54  after crossover move against trade   

                pnl    pv  quote_chg_per_trade  pv_per_trade  
date                                                          
2023-01-27  -12.50k 2.60m                -0.00       -12.50k  
2023-01-30  100.00k 2.70m                 0.04        87.50k  
2023-01-31  -75.00k 2.63m                 0.01        12.50k  
2023-02-01 -100.00k 2.53m                -0.03       -87.50k

In [36]:
for key, self in strategies.items():
    if self._running:
        display(key.upper())
        last_trade = self._summary['trade'].iloc[-1]
        display(self._pnl.query(f"trade == {last_trade}"))

'SFR1'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-02-01      3   -1.00          NaN  95.55      -0.03       0.00   

            take_profit  exit_price notes     pnl     pv  quote_chg_per_trade  \
date                                                                            
2023-02-01         0.00         NaN  None -87.50k 43.75k                -0.03   

            pv_per_trade  
date                      
2023-02-01       -87.50k

'SFR5'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-20      6   -1.00        95.65  95.66      -0.00       0.16   
2023-01-23      6   -1.00          NaN  95.58       0.08       0.16   
2023-01-24      6   -1.00          NaN  95.60      -0.02       0.16   
2023-01-25      6   -1.00          NaN  95.66      -0.06       0.16   
2023-01-26      6   -1.00          NaN  95.62       0.03       0.16   
2023-01-27      6   -1.00          NaN  95.59       0.03       0.03   
2023-01-30      6   -1.00          NaN  95.53       0.06       0.06   
2023-01-31      6   -1.00          NaN  95.58      -0.05       0.06   
2023-02-01      6   -1.00          NaN  95.60      -0.02       0.06   

            take_profit  exit_price notes      pnl      pv  \
date                                                         
2023-01-20         0.16         NaN   NaN   -9.37k 909.38k   
2023-01-23         0.16         NaN   NaN  187.50k   1.10m   
2023-01-24         0.16         NaN   NaN  -50.00k   1.05m   
2023-01-25         0.16         NaN   NaN -137.50k 909.38k   
2023-01-26         0.16         NaN   NaN   87.50k 996.88k   
2023-01-27         0.08         NaN   NaN   75.00k   1.07m   
2023-01-30         0.16         NaN   NaN  137.50k   1.21m   
2023-01-31         0.16         NaN   NaN -125.00k   1.08m   
2023-02-01         0.16         NaN  None  -37.50k   1.05m   

            quote_chg_per_trade  pv_per_trade  
date                                           
2023-01-20                -0.00        -9.37k  
2023-01-23                 0.07       178.13k  
2023-01-24                 0.05       128.13k  
2023-01-25                -0.00        -9.37k  
2023-01-26                 0.03        78.12k  
2023-01-27                 0.06       153.13k  
2023-01-30                 0.12       290.63k  
2023-01-31                 0.07       165.63k  
2023-02-01                 0.05       128.13k

'SFR9'

...

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-26     13   -1.00        96.67  96.67      -0.00       0.09   
2023-01-27     13   -1.00          NaN  96.62       0.05       0.09   
2023-01-30     13   -1.00          NaN  96.61       0.02       0.09   
2023-01-31     13   -1.00          NaN  96.62      -0.02       0.05   
2023-02-01     13   -1.00          NaN  96.64      -0.02       0.05   

            take_profit  exit_price notes     pnl    pv  quote_chg_per_trade  \
date                                                                           
2023-01-26         0.09         NaN   NaN -12.50k 2.75m                -0.00   
2023-01-27         0.09         NaN   NaN 137.50k 2.89m                 0.05   
2023-01-30         0.09         NaN   NaN  37.50k 2.93m                 0.06   
2023-01-31         0.14         NaN   NaN -37.50k 2.89m                 0.05   
2023-02-01         0.14         NaN  None -50.00k 2.84m                 0.03   

            pv_per_trade  
date                      
2023-01-26       -12.50k  
2023-01-27       125.00k  
2023-01-30       162.50k  
2023-01-31       125.00k  
2023-02-01        75.00k

'SFI11'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-26      8   -1.00        96.70  96.70      -0.00       0.08   
2023-01-27      8   -1.00          NaN  96.66       0.05       0.08   
2023-01-30      8   -1.00          NaN  96.64       0.01       0.08   
2023-01-31      8   -1.00          NaN  96.66      -0.01       0.04   
2023-02-01      8   -1.00          NaN  96.67      -0.01       0.04   

            take_profit  exit_price notes     pnl    pv  quote_chg_per_trade  \
date                                                                           
2023-01-26         0.08         NaN   NaN -12.50k 1.75m                -0.00   
2023-01-27         0.08         NaN   NaN 125.00k 1.88m                 0.05   
2023-01-30         0.08         NaN   NaN  25.00k 1.90m                 0.06   
2023-01-31         0.09         NaN   NaN -25.00k 1.88m                 0.05   
2023-02-01         0.09         NaN  None -25.00k 1.85m                 0.03   

            pv_per_trade  
date                      
2023-01-26       -12.50k  
2023-01-27       112.50k  
2023-01-30       137.50k  
2023-01-31       112.50k  
2023-02-01        87.50k

'SFI12'

trade  signal  entry_price  quote  quote_chg  stop_loss  \
date                                                                  
2023-01-26      8   -1.00        96.71  96.72      -0.00       0.08   
2023-01-27      8   -1.00          NaN  96.67       0.04       0.08   
2023-01-30      8   -1.00          NaN  96.67       0.00       0.08   
2023-01-31      8   -1.00          NaN  96.68      -0.01       0.03   
2023-02-01      8   -1.00          NaN  96.69      -0.00       0.03   

            take_profit  exit_price notes     pnl    pv  quote_chg_per_trade  \
date                                                                           
2023-01-26         0.08         NaN   NaN -12.50k 1.94m                -0.00   
2023-01-27         0.08         NaN   NaN 100.00k 2.04m                 0.04   
2023-01-30         0.08         NaN   NaN  12.50k 2.05m                 0.04   
2023-01-31         0.06         NaN   NaN -25.00k 2.03m                 0.03   
2023-02-01         0.06         NaN  None -12.50k 2.01m                 0.03   

            pv_per_trade  
date                      
2023-01-26       -12.50k  
2023-01-27        87.50k  
2023-01-30       100.00k  
2023-01-31        75.00k  
2023-02-01        62.50k